In [ ]:
#jogar db no sql para melhorar as querys

### IMPORT DE BIBLIOTECAS

In [4]:
from collections import defaultdict
import pandas as pd
import numpy as np
import os

import implicit
from implicit.evaluation import train_test_split, ndcg_at_k, AUC_at_k, mean_average_precision_at_k, precision_at_k

from scipy.sparse import csr_matrix, save_npz, load_npz

from matplotlib import pyplot as plt
import seaborn as sns

# from surprise import Dataset, Reader, SVD, SVDpp, KNNWithMeans
# from surprise.model_selection import train_test_split, cross_validate
# from surprise import accuracy
# from surprise.model_selection import cross_validate

# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pickle
# import flask
from tqdm import tqdm
#from deep_translator import GoogleTranslator
import warnings

In [5]:
warnings.filterwarnings('ignore')

### Carregamento de dados de categorias e histórico de clicks

In [11]:
df_de_cat = pd.read_csv(os.getcwd()+'\.txt\category_de.txt') # DataFrame Informativo das categorias

In [12]:
clicks_de = pd.read_csv(os.getcwd()+'\.txt\clicks_de_sample_2.txt', sep = ',', header=0) # df de histórico de clicks
clicks_de.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696606 entries, 0 to 696605
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   UserId       696606 non-null  object
 1   OfferId      696606 non-null  object
 2   OfferViewId  696606 non-null  object
 3   CountryCode  696606 non-null  object
 4   Category     696606 non-null  int64 
 5   Source       696606 non-null  object
 6   UtcDate      696606 non-null  object
 7   Keywords     2886 non-null    object
 8   OfferTitle   693966 non-null  object
dtypes: int64(1), object(8)
memory usage: 47.8+ MB


Valores missing na coluna de OfferTitle que serão excluídos.

In [13]:
clicks_de = clicks_de[clicks_de.OfferTitle.isna() == False]

### Conversão das colunas de usuário e oferta em categórica e criando novas colunas com os códigos adotados 

In [14]:
clicks_de.UserId = clicks_de.UserId.astype('category')
clicks_de.OfferId = clicks_de.OfferId.astype('category')

In [15]:
clicks_de['User'] = clicks_de.UserId.cat.codes
clicks_de['Offer'] = clicks_de.OfferId.cat.codes

### CRIAÇÃO DE COLUNA DE CLICKS POR CATEGORIA

In [16]:
clicks_de.drop(columns = ['Keywords'], axis = 1, inplace=True)  # DROP DAS KEYWORDS

df_de_cat.rename({'Ancertor_ID':'Ancestor_ID'}, axis = 1, inplace = True) # RENAME DO ANCESTOR_ID

df_de_cat.drop('Unnamed: 0', axis =1, inplace = True) # Remoção de coluna Unnamed

clicks_de['Cat_clicks'] = clicks_de.groupby('Category')['OfferId'].transform('count') # Criação de coluna de clicks por categoria

### Merge do Dataframe de categorias com o dataframe de clicks 

In [17]:
clicks_de = clicks_de.merge(df_de_cat, left_on = 'Category', right_on = 'ID').drop(['ID'], axis = 1) 

### Criação de coluna com o nº total de clicks do usuário e nº total de clicks do produto

In [18]:
clicks_de['UserTotalClicks'] = clicks_de.groupby(by=['UserId'])['OfferId'].transform('count')

In [19]:
clicks_de['ProductClicks'] = clicks_de.groupby(by='OfferId')['UserId'].transform('count')

In [69]:
#Cap minimo de clicks para integrar o sistema de recomendação
clicks_de_filtered = clicks_de[(clicks_de.UserTotalClicks > 10) & (clicks_de.ProductClicks > 10)]


### Agrupamento de dos clicks de usuário em ofertas únicas para termos a quantidade de cada usuário em cada oferta.


In [70]:
clicks_per_user_product = clicks_de_filtered.groupby(by=['User','Offer']).count()['UserTotalClicks'].reset_index().rename({'UserTotalClicks':'UserClicks'}, axis = 1)

In [71]:
#clicks_per_user_product.UserClicks[clicks_per_user_product.UserClicks>=1] = 1

### Criação de matrizes esparsas Usuário-item e item-usuário

In [73]:
alpha = 40
sparse_item_user = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['Offer'], clicks_per_user_product['User'])))
sparse_user_item = csr_matrix((clicks_per_user_product['UserClicks'], (clicks_per_user_product['User'], clicks_per_user_product['Offer'])))

data = (sparse_item_user*alpha).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente
sparse_user_item = (sparse_user_item).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente


In [74]:
#Esparsidade de matriz
possible_interactions = sparse_item_user.shape[0]*sparse_item_user.shape[1]
interacted = len(sparse_item_user.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsity

0.9999998492355288

 Mais de 99% das interações possíveis entre usuários e produtos na atual base dados não foi ainda realizada. Segundo artigo: For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. Devemos reavaliar a matriz?

In [76]:
save_npz(os.getcwd()+"/.npz/sparse_user_item.npz", sparse_user_item)
save_npz(os.getcwd()+"/.npz/sparse_item_user.npz", sparse_item_user)

In [77]:
model_path = os.getcwd()+'/.pkl/de_als_model.pkl'

* Criação de diferentes matrizes esparsas para operar com o algoritmo. Usuário-item e item-usuário. Cada uma deve ser usada no momento preciso
* O alfa é o coeficiente de confiabilidade da interação do usuário com um item específico. Valor utilizado fi adotado com base no artigo: https://towardsdatascience.com/alternating-least-square-for-implicit-dataset-with-code-8e7999277f4b. Mas, podemos testar outros valores na validação do modelo.
* Outro artigo de base pra elaboração do modelo: https://medium.com/analytics-vidhya/implementation-of-a-movies-recommender-from-implicit-feedback-6a810de173ac

# FUNÇÃO DE RECOMENDAÇÕES - IMPLICIT

## Treinamento de modelos

In [31]:
offers = pickle.load(open(".pkl/offers.pkl", "rb")) #ET version

In [32]:
alpha = 40
data = (sparse_item_user*alpha).astype('double') # Conversão de tipo para que o modelo ALS funcione corretamente

In [78]:
offers = pickle.load(open(os.getcwd()+"/.pkl/offers.pkl", "rb"))

Carregamentodo dicionário que converte os códigos de ofertas para o seu título de oferta. Ainda falta traduzir do alemão para o inglês para tirar mais significado dos resultados

In [33]:
def als_model():
    
    '''computes p@k and map@k evaluation metrics and saves model'''
    
    #sparse_item_user = load_npz(os.getcwd()+"/.npz/sparse_item_user.npz")
      
    train, test = implicit.evaluation.train_test_split(data, train_percentage=0.8)

    model = implicit.als.AlternatingLeastSquares(factors=100, 
                                                 regularization=0.1, 
                                                 iterations=20, 
                                                 calculate_training_loss=False)
    
    model.fit(train)

    with open(model_path, 'wb') as pickle_out:
        pickle.dump(model, pickle_out)
    
    return train, test, model

In [34]:
train, test, model = als_model()

  0%|          | 0/20 [00:00<?, ?it/s]

In [35]:
implicit.evaluation.precision_at_k(model,train,test)

  0%|          | 0/1873 [00:00<?, ?it/s]

0.12784909614880796

## Avaliação Baseline

In [55]:
def get_k_most_popular(sparse_item_user, k): #k = 10 mais populares, 20 mais populares
    
    """Retorna um array com os códigos dos itens clicados por mais clientes da base de dados"""
    
    new_sparse = sparse_item_user.copy() # Nova esparsa para não modificar a original
    interaction = np.ones_like(new_sparse.data) # Altera os valores da matriz esparsa para '1' se houve interação para que não sejam contadas múltiplas interações do mesmo cliente com o mesmo produto
    new_sparse.data = interaction # Assimila o array de interações de 1's aos dados da matriz esparsa
    pop_items = np.array((new_sparse).sum(axis = 1)).reshape(-1) # Aqui é feita a soma de interações que ocorreram em cada uma das linhas, que correspondem a cada um dos produtos
    most_popular = pd.Series(pop_items).sort_values(ascending = False)[:k] # Os itens são convertidos em série para que possam ser ordenados e então captados os deus com maior incidência para que sejam retornados seus indices(código do produto)
    
    return most_popular.index

 O modelo de baseline considerará como sugestão os produtos mais populares de todo o dataframe como recomendação para todos os clientes

In [56]:
def baseline_precision_at_k(sparse_item_user, k):
    
    """Avalia a precisão caso o modelo recomendasse apenas os itens mais populares a todos os clientes.
    A métrica é calculada com base em uma seleção aleatória de 20% dos clientes."""
    
    # Seleção aleatória de 20% dos usuários para testagem por popularidade
    test_sample = np.random.choice(sparse_item_user.indices,
                                   size = int(0.2*len(sparse_item_user.indices)), 
                                   replace = False) 
    
    most_popular = (get_k_most_popular(sparse_item_user)) # lista de 10 itens mais populares
    
    total_precision = 0 

    for user in test_sample:
        relevance = 0
        for item in most_popular:
            if sparse_item_user[item,user] > 0: # Alteração do pesoa das interações para 0 e 1. Assim, será contabilizado apenas se o usuário interagiu com o item ou não.
                interact = 1
            else:
                interact = 0
            relevance+= interact
        relevance/=k
        total_precision+=relevance
    total_precision/=len(test_sample)
    
    return total_precision

### Avaliação do modelo

In [83]:
def model_evaluation(train, test, model): 
    
    '''Avaliação do modelo treinado com as funções da biblioteca Implicit.
    Retorna dicionário com p@k, map@k, ndcg@k e auc@k.'''

    
    p_at_k = implicit.evaluation.precision_at_k(model, train_user_items=train, 
                                                test_user_items=test,
                                                K=10, 
                                                show_progress = True)
    
    m_at_k = implicit.evaluation.mean_average_precision_at_k(model, 
                                                             train_user_items = train, 
                                                             test_user_items = test, 
                                                             K=10, 
                                                             show_progress = True)

    ndcg_at_k = implicit.evaluation.ndcg_at_k(model, 
                                              train_user_items = train,
                                              test_user_items = test, 
                                              K=10, 
                                              show_progress = True)

    auc_at_k = implicit.evaluation.AUC_at_k(model, 
                                            train_user_items = train, 
                                            test_user_items = test, 
                                            K=10, 
                                            show_progress = True)
    metrics = {'p@K':p_at_k, 
               'map@k': m_at_k, 
               'ndcg@k':ndcg_at_k, 
               'auc@k':auc_at_k}
    
    return metrics

In [84]:
metrics = model_evaluation(train, test, model)

  0%|          | 0/1885 [00:00<?, ?it/s]

  0%|          | 0/1885 [00:00<?, ?it/s]

  0%|          | 0/1885 [00:00<?, ?it/s]

  0%|          | 0/1885 [00:00<?, ?it/s]

In [85]:
metrics

{'p@K': 0.1281985009046265,
 'map@k': 0.05122672119736979,
 'ndcg@k': 0.07384275923453122,
 'auc@k': 0.5541315128824271}

* Sobre metricas de precisão @k: https://medium.com/@m_n_malaeb/recall-and-precision-at-k-for-recommender-systems-618483226c54
* Sobre NDCG: https://towardsdatascience.com/evaluate-your-recommendation-engine-using-ndcg-759a851452d1
* Sobre Mean Average Precision: https://towardsdatascience.com/breaking-down-mean-average-precision-map-ae462f623a52

Ainda não foi realizada qualquer tunagem de hiperaparâmetros. Podemos pegar alguns valores de referencia para rodar um gridsearch

## Funções de recomendações

In [6]:
sparse_user_item = load_npz('sparse_user_item.npz')
sparse_user_item

<416136x325701 sparse matrix of type '<class 'numpy.int64'>'
	with 20434 stored elements in Compressed Sparse Row format>

In [18]:
sparse_user_item.data

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [8]:
sparse_item_user = load_npz('sparse_item_user.npz')
sparse_item_user

<325701x416136 sparse matrix of type '<class 'numpy.float64'>'
	with 20434 stored elements in Compressed Sparse Row format>

In [37]:
list(sparse_item_user.indices)

[154993,
 39584,
 42561,
 58749,
 136474,
 137482,
 168368,
 184609,
 218935,
 278445,
 291246,
 319766,
 329649,
 332255,
 346712,
 357401,
 367426,
 394946,
 402990,
 408647,
 314739,
 362232,
 6031,
 67012,
 152221,
 178831,
 201775,
 204167,
 233326,
 241606,
 261355,
 276326,
 284874,
 285760,
 290749,
 319949,
 365212,
 365577,
 368994,
 378179,
 405545,
 410820,
 414899,
 227951,
 43153,
 109747,
 372582,
 2266,
 118613,
 150977,
 162387,
 167336,
 245902,
 299228,
 300016,
 312634,
 319826,
 340386,
 383216,
 414105,
 215883,
 356488,
 98180,
 98190,
 148394,
 354314,
 163633,
 167663,
 192527,
 369503,
 402003,
 144701,
 127246,
 131142,
 168206,
 229334,
 407343,
 1941,
 19926,
 23831,
 34668,
 69458,
 79176,
 102769,
 111178,
 111987,
 121686,
 134347,
 166205,
 169974,
 173259,
 174832,
 192512,
 220876,
 233278,
 233326,
 236857,
 244040,
 267055,
 289480,
 316398,
 320164,
 324382,
 336258,
 341790,
 342440,
 355242,
 386843,
 406348,
 408113,
 45993,
 94838,
 231452,
 38

In [42]:
model_path = 'de_als_model.pkl'
model_1 = pickle.load(open(model_path, 'rb'))
model_1

In [28]:
def recommend(user):
    
    ''' Retorna uma lista de itens recomendados para o usuário dado de acordo com a biblioteca Implicit.
        Também é retornado uma lista com os itens já clicados por esse usuário'''
    
    sparse_user_item = load_npz("sparse_user_item.npz")
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ = model.recommend(user, sparse_user_item[user])
    
    original_user_items = list(sparse_user_item[user].indices)

    return recommended, original_user_items

In [98]:
user_id = 42563
recommended_items, clicked_items  = recommend(user)

In [99]:
recommended_items

array([ 19258, 316807, 234102, 113209,  22266, 114610, 238298, 221665,
       254879, 133516])

In [100]:
clicked_items

[68, 178227, 181368]

In [46]:
offers = pickle.load(open(".pkl/offers.pkl", "rb")) #ET version

In [48]:
offers

{176956: 'Nova Motors Scooter - Grido 49ccm² 2 stroke',
 309575: 'Nova Motors Scooter - GT3',
 45398: 'Flex Tech Scooter LUNA',
 220024: 'Nova Motors Scooters - Retro Cruisers',
 266402: 'Flex Tech Scooter CITYLEADER',
 258679: 'Nova Motors Scooter - City Star',
 263151: 'Nova Motors Scooters - Energy',
 23367: 'Other Scooter Motoworx Titano 125 ccm 85 km/h',
 309589: 'Easy-Trade Dirt Bike Big Apollo 10Tires Dirtbike CROSS 2012',
 103312: 'Verbatim DVD-R 43538 printable 4.7 GB 120 min. 25 pieces',
 281593: 'HP cleaning cartridge',
 292319: 'Sony MEMORY STICK DUO ADAPTER',
 45497: 'Grundig Steno cassette Steno',
 287395: 'Verbatim BD-R Blu-Ray 50GB 6x Speed \u200b\u200bprintable Jewel Case 1x10',
 83458: 'Intenso BD-RE 2x (25GB) Blu-ray Disc (5201215)',
 217853: 'Verbatim 1x50 Verbatim DVD-R 4',
 64807: 'Intenso CD-R 700 MB 50 Cake Box (1001125)',
 214697: 'HHB - CDR80 Bulk silver T / 50 pieces Thermal Printable',
 216286: 'Verbatim 1x25 Verbatim CD-R 80 / 700MB 52x Speed \u200b\u200bEx

In [66]:
for x in clicked_items:
    print(offers[x])

VOGGENREITER THE BIG BOOK FOR SAXOPHONE
Brille24 Bremen in mottled brown
Potter Babycare Calendula Ointment Wind & Weather 75 ML


In [72]:
def most_similar_items(item_id, n_similar=10):
    '''computes the most similar items'''
    
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)

    similar, score = model.similar_items(item_id, n_similar)

    return similar

In [68]:

list(sparse_user_item.indices)

[42563,
 200549,
 245268,
 112600,
 251066,
 293698,
 7058,
 37198,
 51701,
 243357,
 62654,
 89400,
 207195,
 293446,
 298499,
 94813,
 164189,
 210327,
 166818,
 3863,
 19993,
 24557,
 62014,
 73874,
 87420,
 97921,
 111001,
 120423,
 143318,
 148204,
 184847,
 212109,
 274901,
 313768,
 119456,
 269248,
 114610,
 133516,
 178227,
 307752,
 97057,
 228676,
 245276,
 51092,
 81806,
 175574,
 187110,
 250567,
 261900,
 324703,
 51169,
 153155,
 200321,
 3863,
 19326,
 28069,
 48042,
 58439,
 64239,
 111001,
 135899,
 195417,
 195828,
 204156,
 235338,
 249020,
 261900,
 266285,
 105398,
 104363,
 123740,
 132564,
 53742,
 67419,
 113293,
 114718,
 126730,
 200362,
 274645,
 318377,
 9199,
 9822,
 53498,
 110052,
 179527,
 187565,
 200362,
 200549,
 266322,
 268185,
 1390,
 3863,
 5914,
 9898,
 14893,
 19840,
 19993,
 24557,
 27666,
 65861,
 73874,
 98564,
 120423,
 135242,
 143162,
 145552,
 152050,
 156804,
 163656,
 173054,
 187991,
 207473,
 211426,
 212204,
 220818,
 241694,
 25897

In [97]:
item_id = 42563
most_similar = most_similar_items(item_id, n_similar=10)
most_similar

array([ 42563, 117131, 204705, 268554, 199776, 223799, 159065, 203416,
       269199, 202686])

In [94]:
indices_items = pd.DataFrame(sparse_user_item.indices)
indices_items.columns = ['offerId']
indices_items['Offer_Title'] = indices_items['offerId'].map(offers)

In [95]:
indices_items

,offerId,Offer_Title
0,42563,Leica T
1,200549,IBM ThinkPad T420
2,245268,"AEG ""T76485IH1"" &#39;Heat Pump Dryer&#39;"
3,112600,GeneralKeys digital whiteboard camera for inte...
4,251066,Samsung Galaxy S5 neo black
...,...,...
20429,245596,Edelrid BUD Gray One Size +One Size
20430,95898,Fujitsu Siemens Lifebook S752
20431,109469,VR radio WiFi internet radio with MP3 streamin...
20432,111293,HP Elite Book 8560p


In [96]:
indices_items.to_csv('indice_items.csv',index=False)

In [74]:
for i in most_similar:
    print(offers[i])

hjh OFFICE children's desk chair / children's chair KIDDY GTI-2 fabric black / gray hjh OFFICE
Fireman Sam t-shirt gray for boys in 116
Toppik 3-in-1 Hair Renewal Shampoo
California EEC A+
XOMAX &quot;XM-WH104 XOMAX Flat Screen TV Wall Mount&quot;
Amica cooker with ceramic hob SHC 11552 W
ECO NUK WLAN baby monitor Eco Smart Control 300
Meinrelaxsessel Elderly armchair with stand-up aid KÖLN
WMF_KUECHENMINIS WMF kitchen minis 2-egg cooker
Moet Hennessy Deutschland GmbH Seidlstr. 23 80335 Munich Champagne Moet & Chandon Imperial Rosé Champagne Champagne Brut


In [89]:
itens = list(sparse_user_item.indices)
offer_name = []
for i in itens:
    teste = [offers[i], i]
    offer_name.append(teste)
    
offer_name

[['Leica T', 42563],
 ['IBM ThinkPad T420', 200549],
 ['AEG "T76485IH1" &#39;Heat Pump Dryer&#39;', 245268],
 ['GeneralKeys digital whiteboard camera for interactive presentations',
  112600],
 ['Samsung Galaxy S5 neo black', 251066],
 ['Mae&Mathilda mules Mae&Mathilda beige-snake', 293698],
 ["hjh OFFICE children's desk chair / children's chair KIDDY GTI-2 fabric black / gray hjh OFFICE",
  7058],
 ['WMF_KUECHENMINIS WMF kitchen minis 2-egg cooker', 37198],
 ['Ulla Popken trousers', 51701],
 ['California EEC A+', 243357],
 ['auvisio digital pearl.tv DVB-C2 cable receiver DCR-100.fhd', 62654],
 ['clickandgrill (10.99 €/1kg) Bavarian onion liver cheese', 89400],
 ['Ulla Popken tracks rock', 207195],
 ['Heitronic recessed spotlight GU5', 293446],
 ['Drei Wetter Taft Power (0.96 €/100ml) Hairspray Invisible Power', 298499],
 ['Kenwood smoothie blender', 94813],
 ['Tomato 12 Year Old Highland Single Malt', 164189],
 ['Poland: Danzig / Gdansk', 210327],
 ['Nofly Mosquito Net Insect Screen f

In [91]:
offer_name.to_csv('disponible_offers_name.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [77]:
def most_similar_users(user_id, sparse_user_item,model,n_similar=10):
    '''computes the most similar users'''
#     sparse_user_item = load_npz(os.getcwd()+"/.npz/sparse_user_item.npz")
    
#     with open(model_path, 'rb') as pickle_in:
#         model = pickle.load(pickle_in)

    similar, _ = model.similar_users(user_id, n_similar)

    # original users items
    original_user_items = list(sparse_user_item[user_id].indices)
    
    common_items_users = {}

    # now we want to add the items that a similar user has rated
    for user in similar:
        # Verifica em cada usuário considerado similar quais são os itens que estes
        # tem em comum com o usuário selecionado
        common_items_users[user] = set(list(sparse_user_item[user].indices)) & set(original_user_items)
    
    # retorna usuários similares, e quais são os itens comuns correspondentes a cada um desses usuários
    return similar, common_items_users

In [84]:
similar, common = most_similar_users(user_id,sparse_user_item ,model_1)

In [85]:
similar

array([ 58749, 160232, 279002,  92297,    988,  21897, 160228,  50322,
       317246, 272342])

In [86]:
common

{58749: {68, 178227, 181368},
 160232: {178227},
 279002: {178227},
 92297: {178227},
 988: {178227},
 21897: {178227},
 160228: set(),
 50322: set(),
 317246: set(),
 272342: set()}

In [13]:
def recalculate_user(user_ratings):
    '''adds new user and its liked items to sparse matrix and returns recalculated recommendations
       Receives the user clicked products vector (user_ratings)''' 

    alpha = 40
    m = load_npz('sparse_user_item.npz')
    n_users, n_movies = m.shape

    ratings = [alpha for i in range(len(user_ratings))]

    m.data = np.hstack((m.data, ratings))
    m.indices = np.hstack((m.indices, user_ratings))
    m.indptr = np.hstack((m.indptr, len(m.data)))
    m._shape = (n_users+1, n_movies)

    # recommend N items to new user
    with open(model_path, 'rb') as pickle_in:
        model = pickle.load(pickle_in)
        
    recommended, _ =  zip(*model.recommend(n_users, m, recalculate_user=True))
    
    return recommended

* A matriz m passa a ser a matriz com o novo usuário atualizado e é levada em consideração no para o cálculo de novos vetores.

Nota: 
* Após os ajustes na organização das matrizes esparsas, o modelo parece não mais repetir recomendações de itens que já foram clicados pelo usuário
* O modelo parece também não mais necessitar de tradução dos códigos de ofertas e usuário adotados na matriz esparsa para os códigos da matriz original

### Criação e armazenamento do dicionário código-titulo de oferta (carregado na parte de cima do código).

In [156]:
df_temp = clicks_de[['Offer','OfferTitle']]

In [161]:
df_temp = df_temp.drop_duplicates(['Offer','OfferTitle'])

In [169]:
teste_dicio = dict(zip(df_temp['Offer'], df_temp['OfferTitle']))

In [175]:
pickle.dump(offers, open('.pkl/offers.pkl', 'wb'))

# Testando o codigo feito

In [25]:
user = 289
recommend(user)

ValueError: too many values to unpack (expected 2)